In [ ]:
#install transformers
!pip install transformers

In [ ]:
#install beautiful soup library
!pip install bs4

Import Core Dependancies

In [15]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

Load Summarization Pipeline

In [16]:
summarizer = pipeline('summarization')

Get Blog Post

In [17]:
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [6]:
r = requests.get(URL)

In [18]:
r.text

'<!DOCTYPE html><html lang="en"><head><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);})(window,document,\'script\',\'dataLayer\',\'GTM-WGQVQ44\')</script><script src="https://kit.fontawesome.com/b307ea084c.js" crossorigin="anonymous"></script><meta name="google-site-verification" content="xSsN5mRM5xbHVI00M8uekCnuJ47Da0ER2mYuGhaPM9o"/><link rel="stylesheet" href="/fonts/HackerNoonFont/font-hackernoon.css"/><script>var _iub = _iub || [];\n              _iub.csConfiguration = {"enableCcpa":true,"countryDetection":true,"ccpaAcknowledgeOnDisplay":true,"lang":"en","siteId":1848357,"gdprAppliesGlobally":false,"cookiePolicyId":18778700, "banner":{ "acceptButtonDisplay":true,"customizeButtonDisplay":true,"acceptButtonColor":"#00d500","acceptBut

In [19]:
#scraping website using  beautiful soup
#line below is creating a new instance of beautiful soup and passing in the r.text param and html.parser param
soup = BeautifulSoup(r.text, 'html.parser')
#line below is showing exactly what elements are to be picked
results = soup.find_all(['h1','p'])

In [20]:
#to concatenate into one block of text
text = [result.text for result in results]
ARTICLE = ''.join(text)

In [ ]:
ARTICLE 

Chunk Text

In [ ]:
#we are going to chunk the text because there is a limit to which the hugging face summarizer can summarize

In [22]:
#replacing all exclammation and question  marks with tags
ARTICLE = ARTICLE.replace('.','.<eos>')
ARTICLE = ARTICLE.replace('!','!<eos>')
ARTICLE = ARTICLE.replace('?','?<eos>')
#below we are going to split the sentences at points where the question, exclamation and fullstop were using the tags.
sentences = ARTICLE.split('<eos>')

In [ ]:
sentences

In [24]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
  if len(chunks) == current_chunk +1:
    if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
      chunks[current_chunk].extend(sentence.split(' '))
    else:
      current_chunk += 1
      chunks.append(sentence.split(' '))
  else:
    print(current_chunk)
    chunks.append(sentence.split(' '))

0


In [25]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [ ]:
len(chunks[0].split(' '))

Summarize Text

In [28]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
res

In [ ]:
#to join all the chunks into one body
' '.join([each_chunk['summary_text'] for each_chunk in res])

Outputting Summary to a text file

In [37]:
text = ' '.join([summ['summary_text'] for summ in res])

In [39]:
with open('BlogSummary.txt', 'w') as f:
  f.write(text)